# Arciaga - VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nakhon ratchasima,15.0000,102.1667,21.28,71,17,2.72,TH,1704399479
1,1,ushuaia,-54.8000,-68.3000,9.81,76,100,5.14,AR,1704399480
2,2,margaret river,-33.9500,115.0667,13.20,82,17,5.40,AU,1704399480
3,3,cabo san lucas,22.8909,-109.9124,25.90,42,20,5.81,MX,1704399480
4,4,geraldton,-28.7667,114.6000,24.23,35,0,11.32,AU,1704399480


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
# pip install cartopy geoviews pyproj

In [32]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["City"],
    colormap="viridis",
    title="City Map with Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    size="Humidity",
    frame_width=600,
    frame_height=400,
    tiles="OSM",
    geo=True
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30) &
    (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 20)
]

# Drop any rows with null values
filtered_city_data = criteria_df.dropna()

# Display sample data
filtered_city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nakhon ratchasima,15.0000,102.1667,21.28,71,17,2.72,TH,1704399479
72,72,kurmuk,10.5500,34.2833,23.39,31,0,1.15,SD,1704399494
73,73,mukalla,14.5425,49.1242,22.72,73,2,0.13,YE,1704399494
107,107,sur,22.5667,59.5289,22.22,70,0,0.22,OM,1704399501
110,110,kone,-21.0595,164.8658,26.39,74,0,1.37,NC,1704399501


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,nakhon ratchasima,15.0000,102.1667,21.28,71,17,2.72,TH,1704399479,
1,1,ushuaia,-54.8000,-68.3000,9.81,76,100,5.14,AR,1704399480,
2,2,margaret river,-33.9500,115.0667,13.20,82,17,5.40,AU,1704399480,
3,3,cabo san lucas,22.8909,-109.9124,25.90,42,20,5.81,MX,1704399480,
4,4,geraldton,-28.7667,114.6000,24.23,35,0,11.32,AU,1704399480,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories": "tourism,accommodation",
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nakhon ratchasima - nearest hotel: U-Sabai Park Hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
margaret river - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
geraldton - nearest hotel: Esplanade
laikit, laikit ii (dimembe) - nearest hotel: Gapura Selamat Datang Desa Dimembe
port-aux-francais - nearest hotel: Tour météo
tiksi - nearest hotel: Арктика
yellowknife - nearest hotel: No hotel found
grytviken - nearest hotel: Guano Factory
rio grande - nearest hotel: Hotel Vila Moura Executivo
kinablangan - nearest hotel: Poo Sand Bar Departure Area
mana - nearest hotel: Le Samana
edinburgh of the seven seas - nearest hotel: Sea View
berrahal - nearest hotel: Cimetière chohada
ribeira grande - nearest hotel: Quinta das Bunganvílias
codrington - nearest hotel: Codrington Windfarm Viewing Area
urengoy - nearest hotel: Берлин
ust-nera - nearest hotel: Памятник жертвам трассы «Колыма»
puerto natales - nearest hotel: Hostal Dorotea
b

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,nakhon ratchasima,15.0000,102.1667,21.28,71,17,2.72,TH,1704399479,U-Sabai Park Hotel
1,1,ushuaia,-54.8000,-68.3000,9.81,76,100,5.14,AR,1704399480,Apart Hotel Aires del Beagle
2,2,margaret river,-33.9500,115.0667,13.20,82,17,5.40,AU,1704399480,No hotel found
3,3,cabo san lucas,22.8909,-109.9124,25.90,42,20,5.81,MX,1704399480,Comfort Rooms
4,4,geraldton,-28.7667,114.6000,24.23,35,0,11.32,AU,1704399480,Esplanade
...,...,...,...,...,...,...,...,...,...,...,...
585,585,sajir,25.1825,44.5996,15.39,53,0,3.59,SA,1704399613,السكران
586,586,al hawatah,13.4167,34.6333,25.74,24,0,4.10,SD,1704399613,No hotel found
587,587,touros,-5.1989,-35.4608,28.60,70,2,5.31,BR,1704399326,No hotel found
588,588,santa marta de ortigueira,43.6833,-7.8500,9.03,77,75,5.64,ES,1704399613,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
hotel_city_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],
    colormap="viridis",
    title="City Map with Humidity and Hotel Information",
    xlabel="Longitude",
    ylabel="Latitude",
    size="Humidity",
    frame_width=600,
    frame_height=400,
    tiles="OSM",
    geo=True
)

# Display the map
hotel_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)